## Получение данных

### Импорт библиотек

In [1]:
import pandas as pd # <импорт библиотеки pandas>
from pymystem3 import Mystem

    Прочитаем файл music_project.csv и сохраним его в переменной df.

In [2]:
df = pd.read_csv('/datasets/data.csv')# <чтение файла с данными с сохранением в df>

    Получение первых 10 строк таблицы.

In [3]:
df.head(10) # <получение первых 10 строк таблицы df>

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


    Общая информация о данных таблицы df.

In [4]:
df.info()# <получение общей информации о данных в таблице df>

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [5]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**

Всего в таблице 12 столбцов, тип данных у каждого столбца - float64(2), int64(5), object(5).
Количество значений в столбцах различается. Это говорит о том, что в данных есть пропущенные значения.
Каждая строка таблицы содержит информацию о кредиторе: наличие детей, трудовой стаж и статус, возраст, семейный статус, пол, ежемесячный доход, образование, есть ли долг по кредиту и его цель. Обнаружены сл. проблемы: пропущенные значения в столбцах трудового стажа и дохода, разные регистры в столбце про образование.
При статистическом осмотре через describe() былы выявлены пробдемы с отрицательными занчениями в кол-ве детей и трудовом стаже. Также значения в трудовом стаже слишком большие для подсчета в днях. В столбце с возрастом есть нуливые значения.
Для анализа потребуются столбцы "children", "family_status", "debt", "total_income", "purpose".

## Предобработка данных

### Обработка пропусков

In [6]:
df.isnull().sum() # <суммарное количество пропусков, выявленных методом isnull() в таблице df

df['gender'].value_counts() #проверка уникальных значений 
df['gender'] = df['gender'].replace('XNA', 'F') #замена на наиболее популярный пол

df['total_income'] = df['total_income'].fillna(df.groupby('income_type')['total_income'].transform('mean'))

df['dob_years'] = df.groupby('family_status')['dob_years'].transform(lambda x: x.replace(0, int(x.median())))

df['days_employed'] = abs(df['days_employed']) #приведение значений к абсолютному

med_days_employed = df.groupby('dob_years')['days_employed'].transform('median') # переменные по медиане возраста
df['days_employed'] = df['days_employed'].fillna(med_days_employed) #заменяем пропуски медианой по каждой группе
df.isnull().sum()
df.head(10)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

Данные в столбцах "days_employed" и "total_income" могли быть пропущены как из-за технических ошибок, так и из-за человеческих факторов. Причем данных нет в одних и тех же строках в разных столбцах. Так как из столбца "days_employed" видно, что выгрузка была из разных источников, могло быть такое, что в каком-то из них изначально не было информации о стаже и доходах. 
В столбце с данными о поле некорректные занчения были заменены на наиболее популярный пол. 
Нулевые значения в столбце "dob_years" были заполнены на медианные, сгруппрированной по семейному статусу.
Пропущенные значения в доходах заменены на медианный доход, основываясь на типе занятости.
Отрицательные значения в трудовом стаже быои вощвращены в абсолютное значение по модулю abs(). 

### Замена типа данных

In [8]:
erorr=0
try:
    df['total_income'] = df['total_income'].astype('int') # замена типа данных на целостной
    df['days_employed'] = df['days_employed'].astype('int')
except:
    erorr+=1
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


**Вывод**

Данные в столбце "total_income" были переведены в целые числа при помощи astype(), т.к. при измерении ежемесячного дохода копейками можно пренебречь. Данные были преобразованы с использованием конструкции try-except и метода astype(), т.к. метод to_numeric() преобразовывает данные в float.

### Обработка дубликатов

In [9]:
df.duplicated().sum() # получение суммарного количества дубликатов в таблице df
df = df.drop_duplicates().reset_index(drop=True)  # удаление всех дубликатов из таблицы df специальным методом
#df.duplicated().sum()# проверка на отсутствие полных дублей

df['children'].value_counts() #поиск уникальных значений в столбце 'children'
df['children'] = df['children'].replace(-1, 1) #замена значения -1 на 1 
df = df[df['children'] != 20] #удаление строк, где 20 детей
#df['children'].value_counts() #проверка результата

df['education'].value_counts() #поиск уникальных значений в столбце 'education'
df['education'] = df['education'].str.lower() #приведение к нижнему регистру
#df['education'].value_counts()#проверка результата

df['family_status'].value_counts() #поиск уникальных значений в столбце 'family_status'
df['family_status'] = df['family_status'].str.lower() #приведение к нижнему регистру
#df['family_status'].value_counts()#проверка результата

df['purpose'].unique() #выделение основных причин взятия кредита
df['purpose'] = df['purpose'].replace(43, 'другое')
df['purpose'] = df['purpose'].replace(46, 'другое')
df['purpose'] = df['purpose'].replace(36, 'другое')
df['purpose'] = df['purpose'].replace(58, 'другое')
df['purpose'] = df['purpose'].replace(41, 'другое')
df['purpose'].unique()

df.duplicated().sum() # получение суммарного количества дубликатов в таблице df
df = df.drop_duplicates().reset_index(drop=True)  # удаление всех дубликатов из таблицы df специальным методом

**Вывод**

Обнаружены полные дубликаты, которые могли появиться как вследствие сбоя в записи данных, так и из-за того, что данные были собраны из разных источников. Использовался метод duplicated() для удаления полных дублей. Методом value_counts() были найдены значения в столбцах 'children', 'family_status' и 'education'. В столбце 'children' были найдены артефакты: отрицательное кол-во детей и люди с 20 детьми. Отрицательное кол-во детей было заменено на положительное методом replace(), т.к. это техническая ошибка при выгрузке. Строки, где указано 20 детей были удалены, т.к. нет четкой информации, что там должно стоять "0", "2" или "20". Все значения в столбце с информацией об образовании и семейном статусе были приведены к нижнему регистру методом str.lower(). Были заменены числовые значения в столбце с целями кредита на "другое".

### Лемматизация

In [10]:
df['purpose'].unique() #выделение основных причин взятия кредита


m = Mystem()
def create_category_purpose(row): 
''' постороение функции для лемматизации столбца "purpose"

в качестве аргусента принимает название столбца, в котором нужно сделать лемматищацию,
а выдает новый столбец, разделенный на новые категории
'''
    lem_purpose = m.lemmatize(row['purpose'])
    if 'недвижимость' in lem_purpose:
        return 'недвижимость'
    if 'образование' in lem_purpose:
        return 'образование'
    if 'жилье'  in lem_purpose:
        return 'недвижимость'
    if 'автомобиль'  in lem_purpose:
        return 'автомобиль'
    if 'свадьба'  in lem_purpose:
        return 'свадьба'
    if 'другое'  in lem_purpose:
        return 'другое'
        
df['lem_purpose'] = df.apply(create_category_purpose, axis=1) #применение функции к данным
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lem_purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость


**Вывод**

Сначала методом unique() были найдены уникальные значения для столбца 'purpose'. Цели кредита были лемматизированыи и распределены по сл. категориям:недвижимость, образование, недвижимость, автомобиль и свадьба. Использовалась библиотека  pymystem3. Для удобства была создана функция, которая прописывала одну из категорий в зависимости от леммы.

### Категоризация данных

In [11]:

def children_group(children):
    if children == 0:
        return 'бездетный/ая'
    if 1<= children <= 2:
        return 'малодетный/ая'
    return 'многодетный/ая'

df['income_group'] = pd.qcut(df['total_income'], q = 4, labels = ['низкий', 'средний', 'выше среднего', 'высокий'])


df['children_group'] = df['children'].apply(children_group)
df_family_status = df[['family_status', 'debt']]
df_children = df[['children_group', 'debt']]
df_income = df[['income_group', 'debt']]
df_purpose = df[['lem_purpose', 'debt']]

**Вывод**

## Ответьте на вопросы

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [12]:
df_children = df_children.groupby(['children_group']).agg({'debt':['count','sum']})
df_children['children_result'] = df_children['debt']['sum'] / df_children['debt']['count']
df_children.head()

debt       children_result
                count   sum                
children_group                             
бездетный/ая    14090  1063        0.075444
малодетный/ая    6907   639        0.092515
многодетный/ая    380    31        0.081579

**Вывод**

Наблюдается определенная зависимость между наличием детей и выплатой кредита в срок. Люди без детей и многодетные имеют примерно одинаковый процент должников  - 8% и 7% соответственно. Между тем люди с одним или двумя детьми хуже закрывают кредиты, у них этот процент равен 9%.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [13]:
df_family_status = df_family_status.groupby(['family_status']).agg({'debt':['count','sum']})
df_family_status['family_result'] = df_family_status['debt']['sum'] / df_family_status['debt']['count']
df_family_status.head()

debt      family_result
                       count  sum              
family_status                                  
в разводе               1193   84      0.070411
вдовец / вдова           955   63      0.065969
гражданский брак        4138  385      0.093040
женат / замужем        12290  928      0.075509
не женат / не замужем   2801  273      0.097465

**Вывод**

Хуже всего с выплатой кредита у людей которые не состоят в браке - 10 % из них имеют долги. Далее идут люди состоящие в гражданском браке с 9%, женатые/замужние имеют проблемы с выплатой кредита в 6% случаев. Лучше всего выплачивают люди в разводе или вдовцы/вдовы.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [14]:
df_income = df_income.groupby(['income_group']).agg({'debt':['count','sum']})
df_income['income_result'] = df_income['debt']['sum'] / df_income['debt']['count']
df_income.head()

debt      income_result
              count  sum              
income_group                          
низкий         5345  427      0.079888
средний        5345  471      0.088120
выше среднего  5797  492      0.084871
высокий        4890  343      0.070143

**Вывод**

Уровень дохода влияет на возврат в срок кредита следующим образом: больше всего проблем у группы людей - 12%, которые которые имеют низкий доход, что кажется логичным. Люди со средним доходом и доходом выше среднего имеют следующие проценты должников - 9% и 8% соответственно. Люди с высоким доходом лучше всех возвращают кредит, только 7% имеют долги. Получается, чем больше доход у кредитора, тем лучше он погашает кредит.

### Как разные цели кредита влияют на его возврат в срок?

In [15]:
df_purpose = df_purpose.groupby(['lem_purpose']).agg({'debt':['count','sum']})
df_purpose['purpose_result'] = df_purpose['debt']['sum'] / df_purpose['debt']['count']
df_purpose.head()

debt      purpose_result
              count  sum               
lem_purpose                            
автомобиль     4290  401       0.093473
недвижимость  10775  780       0.072390
образование    3998  369       0.092296
свадьба        2314  183       0.079084

**Вывод**

Одинаковый процент людей испытывает проблемы с выплатой кредита за автомобиль и образование - 9% . За свадьбу и недвижимость 7% не могут погасить кредит вовремя.

## Общий вывод

Наличие детей влияет на способность вернуть кредит вовремя Бездетные лучше всего возвращают кредит, люди с 3я и более детьми склонны возвращать деньги чаще, чем с 1 или 2 детьми. Люди, которые не состоят в отношениях или состоят в неофициальных хуже возвращают кредит, чем замужние.Лучше всего с выплатами у людей в разводе или овдовевших. Чем выше доход, тем больше шансов, что кредит вернут вовремя. Кредит за свадьбу и недвижимость отдают лучше, чем за образоване и автомобиль.